# SUBIR MULTAS

# IMPORTANDO LIBRERIAS

In [1]:
import pandas as pd
import MySQLdb
import time

# CONEXION A LA BASE DE DATOS

In [2]:
db = MySQLdb.connect(host="172.18.55.6",    # tu host, usualmente localhost
                     user="comandato",         # tu usuario
                     passwd="comandato123",  # tu password
                     db="SISTEMECUADOR_ATM")    # el nombre de la base de datos
cur= db.cursor()

# FECHAS DE COMPARACION

### FECHA ACTUAL

In [3]:
Anio_act='2019'
Mes_act='05'
Dia_act='21'

### FECHA ANTERIOR

In [4]:
Anio_ant='2019'
Mes_ant='05'
Dia_ant='20'

# CARACTERISTICA DE LOS ARCHIVOS

### ACTUALES

In [5]:
corr_act={
    'ruta':'..\..\..\..\BASES DATATOOLS\ASIGNACION\\2019\\MAYO\Sistema de cobro_'+Dia_act+'_'+Mes_act+'_'+Anio_act+'.xlsx',
    'hoja':'Asignacion',
    'cartera':'CORRIENTE',
    'identificador':'IDENTIFICACION_DEUDOR'
}

men_act={
    'ruta':'..\..\..\..\BASES DATATOOLS\ASIGNACION\\2019\\MAYO\Sistema de cobro_'+Dia_act+'_'+Mes_act+'_'+Anio_act+'.xlsx',
    'hoja':'Menores',
    'cartera':'MENOR',
    'identificador':'IDENTIFICACION_DEUDOR'
}

conv_act={
    'ruta':'..\..\..\..\BASES DATATOOLS\ASIGNACION\\2019\\MAYO\CONVENIOS_SISTEM_'+Dia_act+'_'+Mes_act+'_'+Anio_act+'.xlsx',
    'hoja':'Hoja1',
    'cartera':'CONVENIOS',
    'identificador':'NUMERO_IDENTIFICACION'
}

dem_act={
    'ruta':'..\..\..\..\BASES DATATOOLS\ASIGNACION\\2019\\MAYO\Sistema de cobro_'+Dia_act+'_'+Mes_act+'_'+Anio_act+'.xlsx',
    'hoja':'Demografico',
    'cartera':'DEMOGRAFICO',
    'identificador':'IDENTIFICACION_DEUDOR'
}

###  ANTERIORES

In [6]:
corr_ant={
    'ruta':'..\..\..\..\BASES DATATOOLS\ASIGNACION\\2019\\MAYO\Sistema de cobro_'+Dia_ant+'_'+Mes_ant+'_'+Anio_ant+'.xlsx',
    'hoja':'Asignacion',
    'cartera':'CORRIENTE',
    'identificador':'IDENTIFICACION_DEUDOR'
}

men_ant={
    'ruta':'..\..\..\..\BASES DATATOOLS\ASIGNACION\\2019\\MAYO\Sistema de cobro_'+Dia_ant+'_'+Mes_ant+'_'+Anio_ant+'.xlsx',
    'hoja':'Menores',
    'cartera':'MENOR',
    'identificador':'IDENTIFICACION_DEUDOR'
}
conv_ant={
    'ruta':'..\..\..\..\BASES DATATOOLS\ASIGNACION\\2019\\MAYO\CONVENIOS_SISTEM_'+Dia_ant+'_'+Mes_ant+'_'+Anio_ant+'.xlsx',
    'hoja':'Hoja1',
    'cartera':'CONVENIOS',
    'identificador':'NUMERO_IDENTIFICACION'
}
dem_ant={
    'ruta':'..\..\..\..\BASES DATATOOLS\ASIGNACION\\2019\\MAYO\Sistema de cobro_'+Dia_ant+'_'+Mes_ant+'_'+Anio_ant+'.xlsx',
    'hoja':'Demografico',
    'cartera':'DEMOGRAFICO',
    'identificador':'IDENTIFICACION_DEUDOR'
}

# LEER ARCHIVOS

In [7]:
arch_corr_act=pd.read_excel(open(corr_act['ruta'],'rb'), sheet_name=corr_act['hoja'], dtype={corr_act['identificador']: str})
arch_corr_ant=pd.read_excel(open(corr_ant['ruta'],'rb'), sheet_name=corr_ant['hoja'], dtype={corr_ant['identificador']: str})
# arch_menr_act=pd.read_excel(open(men_act['ruta'],'rb'), sheet_name=men_act['hoja'], dtype={men_act['identificador']: str})
# arch_menr_ant=pd.read_excel(open(men_ant['ruta'],'rb'), sheet_name=men_ant['hoja'], dtype={men_ant['identificador']: str})
# arch_conv_act=pd.read_excel(open(conv_act['ruta'],'rb'), sheet_name=conv_act['hoja'], dtype={conv_act['identificador']: str})
# arch_conv_ant=pd.read_excel(open(conv_ant['ruta'],'rb'), sheet_name=conv_ant['hoja'], dtype={conv_ant['identificador']: str})

## COLUMNA DE COMPARACION

In [8]:
base_actual = arch_corr_act
base_anterior = arch_corr_ant
cartera_act = corr_act

# base_actual=arch_menr_act
# base_anterior=arch_menr_ant
# cartera_act=men_act

# base_actual=arch_conv_act
# base_anterior=arch_conv_ant
# cartera_act=conv_act



base_anterior['new']='si'
multa_anterior=base_anterior[['NUMERO_CUENTA','new']]
ced_anteriores=base_anterior[['IDENTIFICACION_DEUDOR','new']]

### CRUCE PARA COMPARAR

In [9]:
cruce_multas=pd.merge(base_actual,multa_anterior,on='NUMERO_CUENTA',how='left')

### FILTRO POR VALORES NULOS

In [10]:
nuevas_multas=cruce_multas[cruce_multas['new'].isnull()]

In [11]:
nuevas_multas = nuevas_multas.fillna('')

In [12]:
#subir multas
contador_subidos=0
for row in nuevas_multas.itertuples():
    insert_querry="""INSERT INTO `SISTEMECUADOR_ATM`.`MULTAS_MENSUALES` 
                        (`Cedula`, `No_multa`, `Anio`, `Mes`, `Dia`, 
                         `Dias_mora`, `Saldo`, `Cartera`, `Segmento`, 
                         `Descripcion`, `Placa`) 
                  VALUES('{0}', '{1}', '{2}', '{3}', 
                          '{4}', '{5}', '{6}', '{7}', '{8}',
                          '{9}', '{10}');
    """.format(row.IDENTIFICACION_DEUDOR , row.NUMERO_CUENTA , Anio_act , Mes_act ,
                 Dia_act ,row.DIAS_MORA , row.SALDO , cartera_act['cartera'] , row.SUBCAMPANAPORCUENTA ,
                row.DESCRIPCION_INFRACION,row.PLACA_VEHICULO)
    try:
        insert=cur.execute(insert_querry)
        db.commit()
        contador_subidos=contador_subidos+1
    except:
        db.rollback()
    
print("CANTIDAD DE MULTAS QUE FUERON SUBIDOS : "+str(contador_subidos))

CANTIDAD DE MULTAS QUE FUERON SUBIDOS : 0


In [13]:
info_subidos=0
contador_subidos=0
for row in nuevas_multas.itertuples():
    CEDULA=row.IDENTIFICACION_DEUDOR
    if len(CEDULA)==9 or len(CEDULA)==12 :
        if CEDULA.isdigit():
            CEDULA='0'+CEDULA
    info_querry="""INSERT INTO `SISTEMECUADOR_ATM`.`INFORMACION_CLIENTE` 
                    (`CEDULA`, `TIPO_IDENTIFICACION`, `RAZON_SOCIAL`,`PRIMER_APELLIDO`, 
                     `SEGUNDO_APELLIDO`, `PRIMER_NOMBRE`, `SEGUNDO_NOMBRE`) 
                    VALUES ('{0}', '{1}', '{2}', '{3}', '{4}', '{5}','{6}');
                """.format(CEDULA,row.TIPO_IDENTIFICACION,row.RAZONSOCIAL,
                           row.PRIMER_APELLIDO,row.SEGUNDO_APELLIDO,row.PRIMER_NOMBRE,
                          row.SEGUNDO_NOMBRE)
    try:
        insert=cur.execute(info_querry)
        db.commit()
        info_subidos=info_subidos+1
    except:
        db.rollback()
        
    insert_querry_DB="""INSERT INTO `SISTEMECUADOR_ATM`.`MULTAS` 
                            (`NUMERO_CUENTA`, `ID_CEDULA`, 
                            `SALDO`, `DIAS_MORA`)
                            VALUES ('{0}', '{1}', 
                            '{2}', '{3}');
    """.format(row.NUMERO_CUENTA,CEDULA,row.SALDO,row.DIAS_MORA )
    try:
        insert=cur.execute(insert_querry_DB)
        db.commit()
        contador_subidos=contador_subidos+1
    except:
        db.rollback()
        
    
print("CANTIDAD DE INFRACTORES QUE FUERON SUBIDOS : "+str(info_subidos))
print("CANTIDAD DE MULTAS QUE FUERON SUBIDOS : "+str(contador_subidos))

CANTIDAD DE INFRACTORES QUE FUERON SUBIDOS : 0
CANTIDAD DE MULTAS QUE FUERON SUBIDOS : 0


In [14]:
# INFRACTORES NUEVOS|1|

In [15]:
cruce_cedulas=pd.merge(base_actual,ced_anteriores,on='IDENTIFICACION_DEUDOR',how='left')
nuevas_cedulas=cruce_cedulas[cruce_cedulas['new'].isnull()]
nuevas_cedulas= nuevas_cedulas.fillna('')

In [16]:
nuevo_archivo='NUEVAS CEDULAS CORRIENTE.xlsx'
writer = pd.ExcelWriter(nuevo_archivo, engine='xlsxwriter')
nuevas_cedulas.to_excel(writer, sheet_name='GESTION')
worksheet = writer.sheets['GESTION']
writer.save()

In [17]:
# # base_actual = arch_corr_act
# # base_anterior = arch_corr_ant
# # cartera_act = corr_act

# base_actual=arch_menr_act
# base_anterior=arch_menr_ant
# cartera_act=men_act

# # base_actual=arch_conv_act
# # base_anterior=arch_conv_ant
# # cartera_act=conv_act



# base_anterior['new']='si'
# multa_anterior=base_anterior[['NUMERO_CUENTA','new']]
# ced_anteriores=base_anterior[['IDENTIFICACION_DEUDOR','new']]

In [18]:
# cruce_multas=pd.merge(base_actual,multa_anterior,on='NUMERO_CUENTA',how='left')
# nuevas_multas=cruce_multas[cruce_multas['new'].isnull()]
# nuevas_multas = nuevas_multas.fillna('')

In [19]:
# #subir multas
# contador_subidos=0
# for row in nuevas_multas.itertuples():
#     insert_querry="""INSERT INTO `SISTEMECUADOR_ATM`.`MULTAS_MENSUALES` 
#                         (`Cedula`, `No_multa`, `Anio`, `Mes`, `Dia`, 
#                          `Dias_mora`, `Saldo`, `Cartera`, `Segmento`, 
#                          `Descripcion`, `Placa`) 
#                   VALUES('{0}', '{1}', '{2}', '{3}', 
#                           '{4}', '{5}', '{6}', '{7}', '{8}',
#                           '{9}', '{10}');
#     """.format(row.IDENTIFICACION_DEUDOR , row.NUMERO_CUENTA , Anio_act , Mes_act ,
#                  Dia_act ,row.DIAS_MORA , row.SALDO , cartera_act['cartera'] , row.SUBCAMPANAPORCLIENTE ,
#                 row.DESCRIPCION_INFRACION,row.PLACA_VEHICULO)
#     try:
#         insert=cur.execute(insert_querry)
#         db.commit()
#         contador_subidos=contador_subidos+1
#     except:
#         db.rollback()
    
# print("CANTIDAD DE MULTAS QUE FUERON SUBIDOS : "+str(contador_subidos))

In [20]:
# info_subidos=0
# contador_subidos=0
# for row in nuevas_multas.itertuples():
#     CEDULA=row.IDENTIFICACION_DEUDOR
#     if len(CEDULA)==9 or len(CEDULA)==12 :
#         if CEDULA.isdigit():
#             CEDULA='0'+CEDULA
#     info_querry="""INSERT INTO `SISTEMECUADOR_ATM`.`INFORMACION_CLIENTE` 
#                     (`CEDULA`, `TIPO_IDENTIFICACION`, `RAZON_SOCIAL`,`PRIMER_APELLIDO`, 
#                      `SEGUNDO_APELLIDO`, `PRIMER_NOMBRE`, `SEGUNDO_NOMBRE`) 
#                     VALUES ('{0}', '{1}', '{2}', '{3}', '{4}', '{5}','{6}');
#                 """.format(CEDULA,row.TIPO_IDENTIFICACION,row.RAZONSOCIAL,
#                            row.PRIMER_APELLIDO,row.SEGUNDO_APELLIDO,row.PRIMER_NOMBRE,
#                           row.SEGUNDO_NOMBRE)
#     try:
#         insert=cur.execute(info_querry)
#         db.commit()
#         info_subidos=info_subidos+1
#     except:
#         db.rollback()
        
#     insert_querry_DB="""INSERT INTO `SISTEMECUADOR_ATM`.`MULTAS` 
#                             (`NUMERO_CUENTA`, `ID_CEDULA`, 
#                             `SALDO`, `DIAS_MORA`)
#                             VALUES ('{0}', '{1}', 
#                             '{2}', '{3}');
#     """.format(row.NUMERO_CUENTA,CEDULA,row.SALDO,row.DIAS_MORA )
#     try:
#         insert=cur.execute(insert_querry_DB)
#         db.commit()
#         contador_subidos=contador_subidos+1
#     except:
#         db.rollback()
        
    
# print("CANTIDAD DE INFRACTORES QUE FUERON SUBIDOS : "+str(info_subidos))
# print("CANTIDAD DE MULTAS QUE FUERON SUBIDOS : "+str(contador_subidos))

In [21]:
# cruce_cedulas=pd.merge(base_actual,ced_anteriores,on='IDENTIFICACION_DEUDOR',how='left')
# nuevas_cedulas=cruce_cedulas[cruce_cedulas['new'].isnull()]
# nuevas_cedulas= nuevas_cedulas.fillna('')

In [22]:
# nuevo_archivo='NUEVAS CEDULAS DICCIEMBRE MENORES.xlsx'
# writer = pd.ExcelWriter(nuevo_archivo, engine='xlsxwriter')
# nuevas_cedulas.to_excel(writer, sheet_name='GESTION')
# worksheet = writer.sheets['GESTION']
# writer.save()

In [23]:
# DEMOGRAFICOS NUEVOS

In [24]:
arch_dem_act=pd.read_excel(open(dem_act['ruta'],'rb'), sheet_name=dem_act['hoja'])
arch_dem_ant=pd.read_excel('DEMOGRAFICOS.xlsx', sheet_name='DEMOGRAFICOS')

In [25]:
arch_dem_ant = arch_dem_ant.rename(columns={'DEMO':'TELEFONO'})

In [26]:
arch_dem_act=arch_dem_act[['IDENTIFICACION_DEUDOR','TELEFONO','PRIORIDAD']]
arch_dem_ant=arch_dem_ant[['IDENTIFICACION_DEUDOR','TELEFONO']]

In [27]:
arch_dem_ant['ESTADO']='si'
cruce_demografico=pd.merge(arch_dem_act,arch_dem_ant,on=['IDENTIFICACION_DEUDOR','TELEFONO'],how='left')
nuevas_demograficos=cruce_demografico[cruce_demografico['ESTADO'].isnull()]
nuevas_demograficos= nuevas_demograficos.fillna('')
del nuevas_demograficos['ESTADO']

In [28]:
leer_demografico=pd.read_excel(open('DEMOGRAFICOS.xlsx','rb'), sheet_name='INVALIDO')

In [29]:
campa='DATATOOLS'
leer_demografico['ESTADO']='INCORRECTO'

In [30]:
nuevas_demograficos_2=pd.merge(nuevas_demograficos,leer_demografico,on='TELEFONO',how='left')

In [31]:
def definicion(base):
    tel = str(base['TELEFONO'])
    definicion = 'SIN IDENTIFICAR'
    if (len(tel)==9):
        if (tel[0]=='9'):
            definicion='CELULAR'
    elif (len(tel)==8):
        if tel[1]=='2' or tel[1]=='3' or tel[1]=='4' or tel[1]=='5' or tel[1]=='6' or tel[1]=='7':
            definicion='TELEFONO'
        elif tel[1]=="9" or tel[1]=='8':
            definicion='CELULAR'
    elif (len(tel)==7):
        definicion='TELEFONO'
    elif (len(tel)==6):
        definicion='TELEFONO'
    else:
        definicion = 'SIN IDENTIFICAR'
    return definicion

In [32]:
def arreglarNumero(base):
    tel = str(base['TELEFONO'])
    arreglado = 'SIN IDENTIFICAR'
    if (len(tel)==9):
        if (tel[0]=='9'):
            arreglado='0'+tel
    elif (len(tel)==8):
        if tel[1]=='2' or tel[1]=='3' or tel[1]=='4' or tel[1]=='5' or tel[1]=='6' or tel[1]=='7':
            arreglado='0'+tel
        elif tel[1]=="9" or tel[1]=='8':
            arreglado='09'+tel
    elif (len(tel)==7):
        arreglado='04'+tel
    elif (len(tel)==6):
        arreglado='042'+tel
    else:
        arreglado = 'SIN ARREGLAR'
    return arreglado

In [33]:
def correcto(base):
    tel = str(base['TELEFONO'])
    arreglado = 'INCORRECTO'
    if (len(tel)==9):
        if (tel[0]=='9'):
            arreglado='CORRECTO'
    elif (len(tel)==8):
        if tel[1]=='2' or tel[1]=='3' or tel[1]=='4' or tel[1]=='5' or tel[1]=='6' or tel[1]=='7':
            arreglado='CORRECTO'
        elif tel[1]=="9" or tel[1]=='8':
            arreglado='CORRECTO'
    elif (len(tel)==7):
        arreglado='CORRECTO'
    elif (len(tel)==6):
        arreglado='CORRECTO'
    else:
        arreglado = 'INCORRECTO'
    return arreglado

In [34]:
nuevas_demograficos_2['DEFINICION']=nuevas_demograficos_2.apply(definicion,axis=1)
nuevas_demograficos_2['ARREGLADO']=nuevas_demograficos_2.apply(arreglarNumero,axis=1)
nuevas_demograficos_2['ESTADO']=nuevas_demograficos_2.apply(correcto,axis=1)

In [35]:
nuevas_demograficos_2.head()

,IDENTIFICACION_DEUDOR,TELEFONO,PRIORIDAD,ESTADO,DEFINICION,ARREGLADO
0,911353126,42467634,3,CORRECTO,TELEFONO,042467634
1,924739451,43850418,3,CORRECTO,TELEFONO,043850418
2,300846763,42937938,2,CORRECTO,TELEFONO,042937938
3,1702892348,42280872,3,CORRECTO,TELEFONO,042280872
4,922353875,81589181,,INCORRECTO,SIN IDENTIFICAR,SIN IDENTIFICAR


In [36]:
subir_demograficos=nuevas_demograficos_2[nuevas_demograficos_2['ESTADO']=='CORRECTO']
subir_demograficos.count()

IDENTIFICACION_DEUDOR    540
TELEFONO                 540
PRIORIDAD                540
ESTADO                   540
DEFINICION               540
ARREGLADO                540
dtype: int64

In [37]:
nuevo_archivo='NUEVOS DEMOGRAFICOS.xlsx'
writer = pd.ExcelWriter(nuevo_archivo, engine='xlsxwriter')
subir_demograficos.to_excel(writer, sheet_name='DEMOGRAFICOS')
worksheet = writer.sheets['DEMOGRAFICOS']
writer.save()

In [38]:
info_subidos=0
contador_subidos=0
for row in subir_demograficos.itertuples():
    CEDULA=row.IDENTIFICACION_DEUDOR
    tel = row.ARREGLADO[1:]
    tipo = row.DEFINICION
    insert_querry="""INSERT INTO `TELEFONICOS_NUMEROS`.`TELEFONOS` (`CEDULA`, `NUMERO`, `TIPO`,`CAMPAÑA`)
                   VALUES ('{0}', '{1}', '{2}','{3}');
                """.format(CEDULA,tel,tipo,campa)
    try:
        insert=cur.execute(insert_querry)
        db.commit()
        info_subidos=info_subidos+1
    except:
        db.rollback()
        
    
print("CANTIDAD DE INFRACTORES QUE FUERON SUBIDOS : "+str(info_subidos))
print("CANTIDAD DE MULTAS QUE FUERON SUBIDOS : "+str(contador_subidos))

CANTIDAD DE INFRACTORES QUE FUERON SUBIDOS : 0
CANTIDAD DE MULTAS QUE FUERON SUBIDOS : 0
